In [1]:
import common
import importlib
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import socket

# Show all columns and rows in a dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# graphalytics inserts
data_ins = common.import_gfe("view_graphalytics_inserts").query("omp_threads.isnull() and cluster == 'stones2' and num_threads_read == 0").copy() # data from the experiments
data_ins["build_frequency"].fillna(pd.Timedelta(0), inplace=True) # replace NaT with 0, otherwise the records are ignored in the group by

# graphalytics load
data_load = common.import_gfe("view_graphalytics_load").query("omp_threads.isnull() and cluster == 'stones2' and num_threads_read == 0").copy()
# shape the dataframe as data_ins
data_load["build_frequency"] = pd.Timedelta(0)
data_load["build_frequency_secs"] = np.NaN
data_load["num_snapshots_created"] = 0
data_load["num_threads_write"] = 0

data = pd.concat([data_ins, data_load])
data = data.groupby(["library", "compiler_family", "graph", "build_frequency","num_threads_read", "num_threads_write", "algorithm"]) \
    .agg(completion_time=("median_secs", "median"), count=("median_secs", "count"))
data = data.unstack("algorithm")[("completion_time")]
data.index.set_names("compiler", level=1, inplace=True)

List of available executions:

In [5]:
fn_join_threads = lambda grp: ", ".join(map(str, grp.unique()))
data.reset_index().groupby(["library", "compiler"]).agg(num_threads=("num_threads_write", fn_join_threads))

,,num_threads
library,compiler,
csr,gcc,0
csr-lcc,gcc,0
csr-lcc-numa,gcc,0
csr-numa,gcc,0
g1_v4-bw-sp-ignore-build,gcc,"3, 6, 12"
g1_v4-ref-ignore-build,gcc,"3, 6, 12, 20"
livegraph_ro,gcc,20
llama6,gcc,16
llama6-ref,gcc,16


### CSR

In [8]:
csr = data.loc[("csr", "gcc")].droplevel([1,2,3]).rename(columns = {"lcc": "lcc_std"})
csr["lcc_sorted"] = data.loc[("csr-lcc", "gcc")].droplevel([1,2,3]).rename(columns = {"lcc": "lcc_sorted"})["lcc_sorted"]
csr = csr[["bfs", "cdlp", "lcc_std", "lcc_sorted", "pagerank", "sssp", "wcc"]] # reoder the columns
csr = pd.concat([csr], names=["library"], keys=["csr"]) # prepend the key `csr`

In [9]:
csr

algorithm                 bfs        cdlp      lcc_std  lcc_sorted   pagerank  \
library graph                                                                   
csr     dota-league  1.235302    2.721085   488.513049   70.919549   1.585912   
        graph500-22  1.639669   13.167843   449.729930   19.488179   2.252329   
        graph500-24  2.509866   52.147986  3271.091807  108.531754   5.164838   
        graph500-26  5.025450  110.386501          NaN  704.976753  19.204755   
        uniform-22   1.610202   14.723076     8.904657    2.186826   2.389066   
        uniform-24   2.402618   64.695137    34.872056    6.121123   6.595718   
        uniform-26   4.180822  175.494414   142.640311   22.798893  23.739186   

algorithm                 sssp        wcc  
library graph                              
csr     dota-league   1.732922   1.144053  
        graph500-22   2.988205   1.648705  
        graph500-24   9.470398   3.196715  
        graph500-26  36.034476   9.659982  
        uniform-22    3.528628   1.558210  
        uniform-24   10.972599   3.632904  
        uniform-26   44.953436  10.600717

### Other systems

In [10]:
stinger = pd.concat( [
    data.loc[("stinger5-ref", "gcc", slice(None), slice(None), 0, 40)].droplevel([0, 1, 3, 4, 5])
], names=["library"], keys=["stinger"])
llama = pd.concat( [
    data.loc[("llama6-ref", "gcc", slice(None), "00:00:10", 0, 16)].droplevel([0, 1, 3, 4, 5]) # dynamic scheduling in OpenMP
], names=["library"], keys=["llama"] )
graphone = pd.concat( [
   data.loc[("g1_v4-ref-ignore-build", "gcc", slice(None), slice(None), 0, 12)].droplevel([0, 1, 3, 4, 5])
], names=["library"], keys=["graphone"] )
livegraph = pd.concat( [
    data.loc[("livegraph_ro", "gcc", slice(None), slice(None), slice(None), 20)].droplevel([0, 1, 3, 4, 5])
], names=["library"], keys=["livegraph"] )
competitors = pd.concat([stinger,llama,graphone, livegraph]).rename(columns = {"lcc": "lcc_std"})
competitors["lcc_sorted"] = np.NaN

### Teseo

In [11]:
teseo9 = data.loc[("teseo-ta.9", "gcc")].droplevel([1,2,3]).rename(columns={"lcc": "lcc_std"})
teseo9_dv = data.loc[("teseo-ta-dv.9", "gcc")].droplevel([1,2,3]).rename(columns={"lcc": "lcc_std"})
teseo9_lcc = data.loc[("teseo-ta-lcc.9", "gcc")].droplevel([1,2,3]).rename(columns={"lcc": "lcc_sorted"})
teseo9_lcc_dv = data.loc[("teseo-ta-lcc-dv.9", "gcc")].droplevel([1,2,3]).rename(columns={"lcc": "lcc_sorted"})
teseo_logical = pd.concat([ pd.concat([teseo9, teseo9_lcc["lcc_sorted"]], axis=1 ) ], names=["library"], keys=["teseo_logical"])
teseo_real = pd.concat([ pd.concat([teseo9_dv, teseo9_lcc_dv["lcc_sorted"]], axis=1 ) ], names=["library"], keys=["teseo_real"])

### Dataset

In [12]:
ds = pd.concat([csr, competitors, teseo_logical, teseo_real])
ds = pd.concat([ds, pd.concat([ds.query("library != 'csr'").groupby(by="graph", axis=0).apply(lambda t: t.min())], names=["library"], keys=["best_excl_csr"])])

In [13]:
ds

bfs        cdlp      lcc_std   lcc_sorted  \
library       graph                                                          
csr           dota-league   1.235302    2.721085   488.513049    70.919549   
              graph500-22   1.639669   13.167843   449.729930    19.488179   
              graph500-24   2.509866   52.147986  3271.091807   108.531754   
              graph500-26   5.025450  110.386501          NaN   704.976753   
              uniform-22    1.610202   14.723076     8.904657     2.186826   
              uniform-24    2.402618   64.695137    34.872056     6.121123   
              uniform-26    4.180822  175.494414   142.640311    22.798893   
stinger       dota-league   1.357981    2.009458  1565.042540          NaN   
              graph500-22   1.810404    5.970912  1329.964047          NaN   
              graph500-24   3.133861   26.748108          NaN          NaN   
              graph500-26   5.952022  126.652476          NaN          NaN   
              uniform-22    1.776800    8.584700    17.486793          NaN   
              uniform-24    2.703838   42.902985    77.835743          NaN   
              uniform-26    6.247671  196.284967   343.619172          NaN   
llama         dota-league   1.234896    4.607025          NaN          NaN   
              graph500-22   1.660368   10.968355          NaN          NaN   
              graph500-24   2.747885   50.597313          NaN          NaN   
              uniform-22    1.656119   15.946862  2433.576723          NaN   
              uniform-24    2.523613   90.512970          NaN          NaN   
graphone      dota-league   4.735968    6.365752   623.292636          NaN   
              graph500-22   5.328887   12.121446   599.194614          NaN   
              graph500-24   7.344498   32.765872          NaN          NaN   
              graph500-26  13.273972  190.817887          NaN          NaN   
              uniform-22    5.430410   13.627440    24.452931          NaN   
              uniform-24    7.167861   74.907484   105.851635          NaN   
              uniform-26   15.404325  301.593487   486.778994          NaN   
livegraph     dota-league   1.288784    2.972937   685.458010          NaN   
              graph500-22   2.084764   10.700934   650.491482          NaN   
              graph500-24   4.440983   39.715812          NaN          NaN   
              graph500-26  13.893354  171.140390          NaN          NaN   
              uniform-22    1.812867   15.410497    13.861020          NaN   
              uniform-24    3.095808   41.171111    55.979823          NaN   
              uniform-26    6.828332  283.989918   232.761037          NaN   
teseo_logical dota-league   1.234986    3.301020   758.119155   104.859715   
              graph500-22   1.696823    9.731395   979.369163    76.080428   
              graph500-24   2.670888   42.898176          NaN   648.972667   
              graph500-26   5.566451  198.517443          NaN          NaN   
              uniform-22    1.681496   14.573894    22.921456     4.976576   
              uniform-24    2.541901   63.752128   108.540773    21.534859   
              uniform-26    4.742048  285.140850   495.506611    96.694337   
teseo_real    dota-league   1.223398    2.653245   526.013007    91.008050   
              graph500-22   1.652088    6.398477   530.618496    37.632542   
              graph500-24   2.549679   25.533050          NaN   219.311955   
              graph500-26   4.451635  116.375894          NaN  1532.454444   
              uniform-22    1.592898    8.789830    11.879615     3.503424   
              uniform-24    2.466914   39.406438    47.569604    11.865967   
              uniform-26    4.357165  183.235995   199.325331    47.358692   
best_excl_csr dota-league   1.223398    2.009458   526.013007    91.008050   
              graph500-22   1.652088    5.970912   530.618496    37.632542   
              graph500-24   2.549679   25.533050       

### Create the output file

In [83]:
d0 = ds.loc[("csr", "dota-league")]

In [84]:
path = os.environ["HOME"] + "/workspace/eclipse/paper_gfe/table_graphalytics_auto.tex"

In [85]:
def fmtsecs(value):
    if(value >= 20):
        return "{} s".format(int(value))
    else:
        return "{:.2f} s".format(value)

In [86]:
f = open(path, "w")
f.write(r"""
% This file was autogenerated by the script graphalytics_gen_table.ipynb

\resizebox{\textwidth}{!}{ % the table should use 100% of the available width
    \begin{tabular}[t]{clccccccc|clcccccccc}
    \toprule
        Graph  & System            & BFS       & CDLP       & LCC         & LCC (opt)   & PageRank     & SSSP        & WCC             
      & Graph  & System            & BFS       & CDLP       & LCC         & LCC (opt)   & PageRank     & SSSP        & WCC  \\   
    \midrule
""")

def write_csr(graph0, graph1, name0, name1, num_libraries):
    # first graph
    f.write("\\parbox[t]{2mm}{\\multirow{%s}{*}{\\rotatebox[origin=c]{90}{%s}}} & %% %s \n" % (num_libraries, name0, graph0)) 
    f.write("CSR (baseline) ")
    d0 = ds.loc[("csr", graph0)]
    for c in d0: 
        if np.isnan(c):
            f.write("& DNF")
        else:   
            f.write("& $%s$ " % (fmtsecs(c)))
    f.write(" &\n")
        
    # second graph
    f.write("\\parbox[t]{2mm}{\\multirow{%s}{*}{\\rotatebox[origin=c]{90}{%s}}} & %% %s \n" % (num_libraries, name1, graph1)) 
    f.write("CSR (baseline) ") # without &
    d0 = ds.loc[("csr", graph1)]
    for c in d0: 
        if np.isnan(c):
            f.write("& DNF")
        else:       
            f.write("& $%s$ " % (fmtsecs(c)))
    f.write("\\\\\n")
    
def write_speedup_single0(library, graph):
    completionTime = ds.loc[(library, graph)]
    speedup = completionTime  / ds.loc[("csr", graph)] 
    for column in speedup.index:
        f.write(" & ")
        value = speedup[column]
        best = ds.loc[("best_excl_csr", graph), column]
        if(np.isnan(value)):
            f.write("N/A" if (column == "lcc_sorted" and not library.startswith("teseo")) else "DNF")
        elif completionTime[column] <= best:
            f.write("$\winner{");
            f.write("{:.2f}x".format(value))
            f.write("}$")
        else:
            f.write("${:.2f}x$".format(value))
    
def write_speedup_single(label, library, graph0, graph1):
    f.write(" & " + label)
    write_speedup_single0(library, graph0)
    f.write(" & & " + label)
    write_speedup_single0(library, graph1)
    f.write(" \\\\ \n")
    
def write_speedup(graph0, graph1, include_llama = True):
    write_speedup_single("Stinger", "stinger", graph0, graph1)
    if(include_llama): write_speedup_single("LLAMA", "llama", graph0, graph1)
    write_speedup_single("GraphOne", "graphone", graph0, graph1)
    write_speedup_single("LiveGraph", "livegraph", graph0, graph1)
    write_speedup_single("Teseo, log. vtx", "teseo_logical", graph0, graph1)
    write_speedup_single("Teseo, real vtx", "teseo_real", graph0, graph1)
    

write_csr("dota-league", "graph500-22", "DOTA League", "Graph500 SF 22", 7)
write_speedup("dota-league", "graph500-22")
f.write("\\midrule \n")
write_csr("uniform-24", "graph500-24", "Uniform SF 24", "Graph500 SF 24", 7)
write_speedup("uniform-24", "graph500-24")
f.write("\\midrule \n")
write_csr("uniform-26", "graph500-26", "Uniform SF 26", "Graph500 SF 26", 6)
write_speedup("uniform-26", "graph500-26", include_llama = False)
    
f.write(r"""
        \bottomrule
    \end{tabular}
} % end fo resizebox
""")
f.close()